# Playing TextWorld generated games with OpenAI Gym
This tutorial shows how to play a text-based adventure game **generated by TextWorld** using OpenAI's Gym API.

### Generate a new TextWorld game

In [1]:
!tw-make custom --world-size 2 --quest-length 3 --nb-objects 10 --output tw_games/test_game.ulx -f -v --seed 1234

Global seed: 1234
Game generated: /home/avnerus/Projects/Marrow/rl/tw_notebooks/tw_games/test_game.ulx

Objective:
It's time to explore the amazing world of TextWorld! Here is how to play! First of all, go east. After that, recover the type 4 key from the non-euclidean locker in the attic. After that, lock the type 4 chest in the attic. Got that? Good!

Walkthrough:
go east > take type 4 key from non-euclidean locker > lock type 4 chest with type 4 key

-= Stats =-
Nb. locations: 2
Nb. objects: 10


### Register the game with Gym
In order to call to `gym.make`, we need to create a valid `env_id` for our game.

In [2]:
import textworld.gym
env_id = textworld.gym.register_game('tw_games/test_game.ulx')

### Make the gym environment
With our `env_id` we are ready to use gym to start the new environment.

In [3]:
import gym
env = gym.make(env_id)

### Start the game
Like for other Gym environments, we start a new game by calling the `env.reset` method. It returns the initial observation text string as well as a dictionary for additional informations (more on that later).

In [4]:
obs, infos = env.reset()
print(obs)




                    ________  ________  __    __  ________        
                   |        \|        \|  \  |  \|        \       
                    \$$$$$$$$| $$$$$$$$| $$  | $$ \$$$$$$$$       
                      | $$   | $$__     \$$\/  $$   | $$          
                      | $$   | $$  \     >$$  $$    | $$          
                      | $$   | $$$$$    /  $$$$\    | $$          
                      | $$   | $$_____ |  $$ \$$\   | $$          
                      | $$   | $$     \| $$  | $$   | $$          
                       \$$    \$$$$$$$$ \$$   \$$    \$$          
              __       __   ______   _______   __        _______  
             |  \  _  |  \ /      \ |       \ |  \      |       \ 
             | $$ / \ | $$|  $$$$$$\| $$$$$$$\| $$      | $$$$$$$\
             | $$/  $\| $$| $$  | $$| $$__| $$| $$      | $$  | $$
             | $$  $$$\ $$| $$  | $$| $$    $$| $$      | $$  | $$
             | $$ $$\$$\$$| $$  | $$| $$$$$$$\| $$      | $

### Interact with the game
The `env.step` method is used to send text command to the game. This method returns the observation for the new state, the current game score, whether the game is done, and dictionary for additional informations (more on that later).

In [6]:
obs, score, done, infos = env.step("open safe")
print(obs)

You open the safe.




### Make a simple play loop
We now have everything we need in order to interactively play a text-based game.

In [7]:
try:
    done = False
    obs, _ = env.reset()
    print(obs)
    nb_moves = 0
    while not done:
        command = input("> ")
        obs, score, done, _ = env.step(command)
        print(obs)
        nb_moves += 1

except KeyboardInterrupt:
    pass  # Press the stop button in the toolbar to quit the game.

print("Played {} steps, scoring {} points.".format(nb_moves, score))




                    ________  ________  __    __  ________        
                   |        \|        \|  \  |  \|        \       
                    \$$$$$$$$| $$$$$$$$| $$  | $$ \$$$$$$$$       
                      | $$   | $$__     \$$\/  $$   | $$          
                      | $$   | $$  \     >$$  $$    | $$          
                      | $$   | $$$$$    /  $$$$\    | $$          
                      | $$   | $$_____ |  $$ \$$\   | $$          
                      | $$   | $$     \| $$  | $$   | $$          
                       \$$    \$$$$$$$$ \$$   \$$    \$$          
              __       __   ______   _______   __        _______  
             |  \  _  |  \ /      \ |       \ |  \      |       \ 
             | $$ / \ | $$|  $$$$$$\| $$$$$$$\| $$      | $$$$$$$\
             | $$/  $\| $$| $$  | $$| $$__| $$| $$      | $$  | $$
             | $$  $$$\ $$| $$  | $$| $$    $$| $$      | $$  | $$
             | $$ $$\$$\$$| $$  | $$| $$$$$$$\| $$      | $

## Request additional information
<span style="color:red">_*Only available for games generated with TextWorld._</span>

To ease the learning process of AI agents, TextWorld offers control over what information should be available alongside the game's narrative (i.e. the observation).

Let's request the list of __admissible__ commands (i.e. commands that are guaranteed to be understood by the game interpreter) for every game state. We will also request the list of entity names that can be interacted with. For the complete list of information that can be requested, see the [documentation](https://textworld.readthedocs.io/en/latest/_modules/textworld/envs/wrappers/filter.html?highlight=envinfos)).

In [9]:
import textworld
request_infos = textworld.EnvInfos(admissible_commands=True, entities=True)

# Requesting additional information should be done when registering the game.
env_id = textworld.gym.register_game('tw_games/test_game.ulx', request_infos)
env = gym.make(env_id)

obs, infos = env.reset()
print("Entities: {}".format(infos["entities"]))
print("Admissible commands:\n  {}".format("\n  ".join(infos["admissible_commands"])))

Entities: ['type 4 chest', 'non-euclidean locker', 'safe', 'key', 'sock', 'workbench', 'type 4 key', 'non-euclidean passkey', 'top hat', 'insect', 'north', 'south', 'east', 'west']
Admissible commands:
  drop insect
  examine insect
  examine key
  examine safe
  go east
  inventory
  look
  open safe
  take key
